# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/Weather_Data_04132021.csv")
weather_df

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Windspeed (mph),Country,Date
0,harbour breton,47.4833,-55.7983,40.32,78,99,28.21,CA,1618355284
1,punta arenas,-53.1500,-70.9167,50.00,87,100,9.22,CL,1618355240
2,hobart,-42.8794,147.3294,61.00,51,40,21.85,AU,1618355285
3,bandarbeyla,9.4942,50.8122,79.03,75,47,8.16,SO,1618355285
4,rikitea,-23.1203,-134.9692,77.68,66,11,7.92,PF,1618355232
...,...,...,...,...,...,...,...,...,...
560,aripuana,-9.1667,-60.6333,75.34,91,69,0.60,BR,1618355621
561,albert,50.0009,2.6510,39.99,75,0,4.61,FR,1618355622
562,kalmunai,7.4167,81.8167,77.99,84,97,5.35,LK,1618355622
563,mundra,22.8500,69.7333,78.12,48,0,8.55,IN,1618355623


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
from api_keys import (weather_api_key, g_key)

In [4]:
gmaps.configure(api_key = g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations as a float
locations = weather_df[["Latitude", "Longitude"]].astype(float)


In [6]:
weather_df['Humidity'] = weather_df['Humidity'].astype(float)

In [7]:
#verify that Humidity is now a float
weather_df.dtypes

City Name           object
Latitude           float64
Longitude          float64
Max Temp           float64
Humidity           float64
Cloudiness           int64
Windspeed (mph)    float64
Country             object
Date                 int64
dtype: object

In [8]:
humidity = weather_df['Humidity']
humidity

0      78.0
1      87.0
2      51.0
3      75.0
4      66.0
       ... 
560    91.0
561    75.0
562    84.0
563    48.0
564    74.0
Name: Humidity, Length: 565, dtype: float64

In [10]:
# Create a Humidity Heatmap layer
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#* A max temperature lower than 80 degrees but higher than 70.
# * Wind speed less than 10 mph.
#  * Zero cloudiness.
#  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
perfect_weather = weather_df.loc[(weather_df['Max Temp'] > 70) & (weather_df['Max Temp'] < 80) & 
                                 (weather_df['Windspeed (mph)'] < 10) & (weather_df['Cloudiness'] == 0)]
perfect_weather = perfect_weather.reset_index()
del perfect_weather['index']
perfect_weather.at[0,'Country']="NM"
perfect_weather

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Windspeed (mph),Country,Date
0,luderitz,-26.6481,15.1594,78.49,33.0,0,7.70,NM,1618355296
1,salalah,17.0151,54.0924,78.80,73.0,0,3.44,OM,1618355316
2,sao filipe,14.8961,-24.4956,77.65,58.0,0,7.52,CV,1618355333
3,mount isa,-20.7333,139.5000,75.20,23.0,0,4.61,AU,1618355346
4,lumding,25.7535,93.1732,70.03,42.0,0,1.68,IN,1618355366
5,madera,37.1666,-119.8343,71.01,35.0,0,1.99,US,1618355120
6,caravelas,-17.7125,-39.2481,75.47,78.0,0,5.75,BR,1618355403
7,neuquen,-38.9516,-68.0591,77.00,44.0,0,3.44,AR,1618355197
8,pilar,-9.5972,-35.9567,77.00,88.0,0,3.44,BR,1618355459
9,san martin,-33.0810,-68.4681,71.60,49.0,0,5.97,AR,1618355478


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
perfect_weather["Hotel Name"] = ""
hotel_df = perfect_weather
hotel_df

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Windspeed (mph),Country,Date,Hotel Name
0,luderitz,-26.6481,15.1594,78.49,33.0,0,7.70,NM,1618355296,
1,salalah,17.0151,54.0924,78.80,73.0,0,3.44,OM,1618355316,
2,sao filipe,14.8961,-24.4956,77.65,58.0,0,7.52,CV,1618355333,
3,mount isa,-20.7333,139.5000,75.20,23.0,0,4.61,AU,1618355346,
4,lumding,25.7535,93.1732,70.03,42.0,0,1.68,IN,1618355366,
5,madera,37.1666,-119.8343,71.01,35.0,0,1.99,US,1618355120,
6,caravelas,-17.7125,-39.2481,75.47,78.0,0,5.75,BR,1618355403,
7,neuquen,-38.9516,-68.0591,77.00,44.0,0,3.44,AR,1618355197,
8,pilar,-9.5972,-35.9567,77.00,88.0,0,3.44,BR,1618355459,
9,san martin,-33.0810,-68.4681,71.60,49.0,0,5.97,AR,1618355478,


In [ ]:
hotels = []
counter = 0

for hotel in hotels:
    hotel_name = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    counter = counter + 1

In [38]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# geocoordinates
locations = perfect_weather[["Latitude","Longitude"]].astype(float)

target_coordinates = locations
target_radius = 3.106856
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in types_df.iterrows():

    # get restaurant type from df
    hotel_name = row['hotel']

    # add keyword to params dict
    params['keyword'] = name

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
        hotel_df.loc[index, 'name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        




NameError: name 'types_df' is not defined

In [36]:
locations

,Latitude,Longitude
0,-26.6481,15.1594
1,17.0151,54.0924
2,14.8961,-24.4956
3,-20.7333,139.5000
4,25.7535,93.1732
5,37.1666,-119.8343
6,-17.7125,-39.2481
7,-38.9516,-68.0591
8,-9.5972,-35.9567
9,-33.0810,-68.4681


In [34]:
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
